## Import dependent libraries

In [1]:
import math
import random
import sys
import time
import numpy
from collections import deque, Counter
from pandas import read_csv
from matplotlib import pyplot
from numpy import concatenate, array, asarray
from pandas import DataFrame
from pandas import concat
from keras import optimizers
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.externals import joblib
from keras.models import Model, model_from_json
from keras.layers import Dense, Dropout, BatchNormalization, Activation, Input
from keras.layers.recurrent import LSTM
from keras.utils import to_categorical, plot_model
from keras.callbacks import ReduceLROnPlateau
import pyaudio
import aubio

Using TensorFlow backend.


In [2]:
def backspace(n):
    # print((b'\x08').decode(), end='')
    print('\r', end='')

In [3]:
def print_in_place(string):
    s = str(string)                       
    print(s, end='')                        
    sys.stdout.flush()                    
    backspace(len(s) +1)  
    time.sleep(0.01)

## Convert series to supervised learning

In [4]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j + 1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j + 1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j + 1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

## Encode and Scale the values

In [5]:
# specify the number of lag hours
n_hours = 10
n_features = 2
n_obs = n_hours * n_features
# normalize features
scaler = joblib.load('scaler.pkl')

In [ ]:
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

In [ ]:
loaded_model.summary()

In [ ]:
def process_array_for_prediction(testx):
    testx = DataFrame(data = testx)
    testx = testx.values
    testx = testx.astype('float32')
    testx = scaler.transform(testx)
    testx = series_to_supervised(testx, n_hours, 1)
    testx = testx.values
    testx = testx[:, :n_obs]
    testx = testx.reshape((testx.shape[0], n_hours, n_features))
    return testx

In [ ]:
testx = [[60, 25456],
[59, 14880],
[62, 25820],
[60, 21361],
[59, 21153],
[60, 26320],
[58, 20404],
[57, 86203],
[59, 20645],
[57, 19400],
[60, 26310]]
input_values = testx
testx = process_array_for_prediction(testx)

In [ ]:
prediction = loaded_model.predict(testx)
prediction = asarray(prediction).ravel().reshape(-1,2)

In [ ]:
prediction = scaler.inverse_transform(prediction)
prediction = prediction.astype('int')

In [ ]:
print("INPUT_VALUES:")
print(input_values)
print("PREDICTION:")
print(prediction[0])
print("Note:", prediction[0][0], "Length:", prediction[0][1])

In [ ]:
for i in range(101):
    print_in_place(str(i))

In [ ]:
generator = Generator()
generator.generate_set()

In [37]:
x = 7

In [41]:
print(x)

12348


In [40]:
x = x *6 * 7

In [36]:
print(x)

12348
